In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, Tool
from duckduckgo_search import DDGS
from langchain_core.prompts import ChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain_core.messages import AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import os
from dotenv import load_dotenv

In [2]:
load_dotenv("secrets.env")
my_key = os.getenv("GEMINI_KEY")

os.environ["GOOGLE_API_KEY"] = my_key
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

In [3]:
TRUSTED_SITES = [
    "cdc.gov", 
    "nih.gov",
    "drugs.com",
    "medlineplus.gov",
    "hopkinsmedicine.org",
    "who.int",
    "fda.gov",
    "health.harvard.edu"
]

In [4]:
def medical_search(query: str, max_results: int = 5) -> str:
    filter_domains = " OR ".join([f"site:{domain}" for domain in TRUSTED_SITES])
    full_query = f"{query} {filter_domains}"

    search = SerpAPIWrapper()

    try:
        raw_results = search.results(full_query)

        results_text = []
        count = 0

        for result in raw_results.get("organic_results", []):
            url = result.get("link", "")
            if any(domain in url for domain in TRUSTED_SITES):
                title = result.get("title", "")
                snippet = result.get("snippet", "")
                results_text.append(f"Title: {title}\nSnippet: {snippet}\nSource: {url}")
                count += 1
            if count >= max_results:
                break

        return "\n\n".join(results_text) if results_text else "No trusted medical sources found."

    except Exception as e:
        return f"Search failed: {str(e)}"

In [5]:
tools = [
    Tool(
        name="TrustedMedicalSearch",
        func=medical_search,
        description="Searches medical information from trusted sources like CDC, NIH, Mayo Clinic, and more."
    )
]

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [("system", (
        "You are a medical assistant. You have access to a specialized web search tool "
        "that ONLY returns information from a predefined list of trustworthy sources, "
        "including reputable general, academic, and medical websites. "
        "Use this tool when you need to find current or specific information. "
        "When you provide an answer based on information from the search tool, "
        "If the tool returns 'No information found from trusted sources', state that clearly and do not invent information."
        )),
        ("user", "{input}"),
        AIMessage(content="Okay, I will use the trusted search tool if necessary to find the answer from reliable sources."),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [7]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [8]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True)

In [9]:
user_input = "What is Ozempic?"

In [10]:
response = agent_executor.invoke({"input": user_input})
print(f"{response['output']}") 



> Entering new AgentExecutor chain...

Invoking: `TrustedMedicalSearch` with `Ozempic`


Title: Ozempic: Uses, Dosage, Side Effects, Warnings
Snippet: Ozempic (semaglutide) is a prescription medication used for adults with type 2 diabetes to lower their blood sugar levels (A1C).
Source: https://www.drugs.com/ozempic.html

Title: FDA's Concerns with Unapproved GLP-1 Drugs Used for ...
Snippet: FDA is aware of counterfeit Ozempic marketed in the U.S. Counterfeit drugs claim to be authentic, but could contain the wrong ingredients, ...
Source: https://www.fda.gov/drugs/postmarket-drug-safety-information-patients-and-providers/fdas-concerns-unapproved-glp-1-drugs-used-weight-loss

Title: highlights of prescribing information - accessdata.fda.gov
Snippet: Patients receiving OZEMPIC in combination with an insulin secretagogue (e.g., sulfonylurea) or insulin may have an increased risk of hypoglycemia, including ...
Source: https://www.accessdata.fda.gov/drugsatfda_docs/label/2025/209637s025

In [11]:
if 'intermediate_steps' in response and response['intermediate_steps']:
    print("\n--- Agent's Intermediate Steps (Context & Raw Tool Output) ---")
    for step_number, (action, observation) in enumerate(response['intermediate_steps']):
        print(f"Step {step_number + 1}:")
        print(f"  Action Tool: {action.tool}")
        print(f"  Action Input: {action.tool_input}")
        print(f"  Observation (Raw Tool Output):\n{observation}\n---")


--- Agent's Intermediate Steps (Context & Raw Tool Output) ---
Step 1:
  Action Tool: TrustedMedicalSearch
  Action Input: Ozempic
  Observation (Raw Tool Output):
Title: Ozempic: Uses, Dosage, Side Effects, Warnings
Snippet: Ozempic (semaglutide) is a prescription medication used for adults with type 2 diabetes to lower their blood sugar levels (A1C).
Source: https://www.drugs.com/ozempic.html

Title: FDA's Concerns with Unapproved GLP-1 Drugs Used for ...
Snippet: FDA is aware of counterfeit Ozempic marketed in the U.S. Counterfeit drugs claim to be authentic, but could contain the wrong ingredients, ...
Source: https://www.fda.gov/drugs/postmarket-drug-safety-information-patients-and-providers/fdas-concerns-unapproved-glp-1-drugs-used-weight-loss

Title: highlights of prescribing information - accessdata.fda.gov
Snippet: Patients receiving OZEMPIC in combination with an insulin secretagogue (e.g., sulfonylurea) or insulin may have an increased risk of hypoglycemia, including ...
So